In [72]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dropout, Dense, Flatten
from tensorflow.keras import Sequential
import numpy as np
import pandas as pd
import random

In [73]:
# Creating the dataset. 
p = 100
a = []
for i in range(p+1):
    temp = []
    for j in range(p+1):
        if(j == i):
            temp += [1]
        else:
            temp += [0]
    a += [temp]
a = np.array(np.array(a))

In [74]:
x = a[-2]
y = a[-1]

In [75]:
# generating the training set
data_sets = []
labels = []
num_sets = 10000
for i in range(num_sets//2):
    # generate random nums
    idx = [random.randint(0, p-2) for i in range(p-1)]
    idx.sort()
    temp_set = a[idx]
    temp_set = np.append([x], temp_set, axis =0)
    data_sets += [temp_set]
    labels += [0]
for i in range(num_sets//2):
    # generate random nums
    idx = [random.randint(0, p-2) for i in range(p-1)]
    idx.sort()
    temp_set = a[idx]
    temp_set = np.append([y], temp_set, axis =0)
    data_sets += [temp_set]
    labels += [1]
data_sets = np.array(data_sets)
labels = np.array(labels)

# np.random.shuffle(data_sets) # Now the dataset is randomly shuffled

In [76]:
def unison_shuffle(a, b):
    idx = np.random.permutation(len(a))
    return a[idx], b[idx]
data_sets, labels = unison_shuffle(data_sets, labels)

In [77]:
train_frac = int(num_sets*0.7)
test_frac = num_sets - train_frac
x_train, y_train = data_sets[:train_frac], labels[:train_frac]
x_test, y_test = data_sets[:test_frac], labels[:test_frac]

In [78]:
x_train.shape

(7000, 100, 101)

In [79]:
x_test.shape

(3000, 100, 101)

In [80]:
y_train.shape

(7000,)

In [81]:
y_test.shape

(3000,)

In [82]:
# generated many possible training sequences
idx_set = random.randint(0, 9999)
training_instance = data_sets[idx_set]
pd.DataFrame(training_instance)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [83]:
hidden_layers = 150

In [84]:
training_instance.shape

(100, 101)

In [85]:
model = Sequential()
model.add(tf.keras.Input(shape=training_instance.shape))
model.add(SimpleRNN(units=hidden_layers, return_sequences=False, activation='tanh'))
model.add(Dense(2, activation = 'softmax'))

In [86]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (None, 150)               37800     
                                                                 
 dense_8 (Dense)             (None, 2)                 302       
                                                                 
Total params: 38,102
Trainable params: 38,102
Non-trainable params: 0
_________________________________________________________________


In [87]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x_train, y_train, epochs=100, callbacks=callback, validation_split=0.2)

Epoch 1/100
175/175 [==============================] - 3s 15ms/step - loss: 0.1137 - accuracy: 0.9346 - val_loss: 7.2330e-04 - val_accuracy: 1.0000
Epoch 2/100
175/175 [==============================] - 2s 14ms/step - loss: 4.6291e-04 - accuracy: 1.0000 - val_loss: 3.0143e-04 - val_accuracy: 1.0000
Epoch 3/100
175/175 [==============================] - 3s 14ms/step - loss: 2.2267e-04 - accuracy: 1.0000 - val_loss: 1.6733e-04 - val_accuracy: 1.0000
Epoch 4/100
175/175 [==============================] - 3s 14ms/step - loss: 1.3298e-04 - accuracy: 1.0000 - val_loss: 1.0746e-04 - val_accuracy: 1.0000
Epoch 5/100
175/175 [==============================] - 3s 15ms/step - loss: 8.9023e-05 - accuracy: 1.0000 - val_loss: 7.4998e-05 - val_accuracy: 1.0000
Epoch 6/100
175/175 [==============================] - 3s 15ms/step - loss: 6.3969e-05 - accuracy: 1.0000 - val_loss: 5.5434e-05 - val_accuracy: 1.0000
Epoch 7/100
175/175 [==============================] - 3s 15ms/step - loss: 4.8212e-05 - acc

In [88]:
model.evaluate(x_test, y_test)

94/94 [==============================] - 1s 6ms/step - loss: 7.9473e-11 - accuracy: 1.0000


[7.947285191045594e-11, 1.0]